In [1]:
from src.data._dataobject import DataObject
from src.data.types._date import NSDate
from src.data.types._datetime import DateTime
from src.data.types._text import String, NSString
from src.data.types._int import Int
from src.data.types._phone_number import PhoneNumber
from src.data.types._id import MongoId, Id
from src.data.types._time import Time

In [2]:

class User(DataObject):
    first_name = NSString(min_length=3, json_field='firstName')
    last_name = NSString(json_field='lastName')
    mobile = PhoneNumber()
    id = Id()
    created = DateTime(default='now')
    birthday=NSDate()
    favourite_time=Time()

In [3]:
u = User(
    first_name='Frodo',
    last_name='Baggins',
    mobile='+39 375 57 44 512',
    birthday='1988-07-25',
    favourite_time='18:00',
)

In [4]:
User.alchemy_table()

Table('user', MetaData(), Column('birthday', Date(), table=<user>, nullable=False), Column('created', DateTime(), table=<user>, default=ColumnDefault(DateTime(2022, 2, 12, 9, 56, 13, 409142, tzinfo=Timezone('Europe/Berlin')))), Column('favourite_time', Time(), table=<user>), Column('first_name', String(), CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x00000258AD894880>), table=<user>, nullable=False), Column('id', Integer(), table=<user>, primary_key=True), Column('last_name', String(), table=<user>, nullable=False), Column('mobile', String(), table=<user>), schema=None)

In [5]:
dfs = User.to_df([u, u, u])

In [6]:
import pandas as pd

In [7]:
pd.DataFrame([{'a': 1, 'b': 2},{'a': 1, 'b': 2},{'a': 1, 'b': 2}])

,a,b
0,1,2
1,1,2
2,1,2


In [8]:
print(User.to_ts())

class User {
  birthday: any
  created: any
  favourite_time: any
  first_name: string
  id: number
  last_name: string
  mobile: string
}


In [9]:
User.from_df(dfs)

[<User id=None>, <User id=None>, <User id=None>]

In [10]:
from tornado.web import Application
from tornado.ioloop import IOLoop
from tornado.auth import GoogleOAuth2Mixin, OpenIdMixin
app = Application([User.to_api()])
app.listen(8888)
# IOLoop.current().start()

In [11]:
class H:
    def __contains__(self, x):
        print(x)
        return False

In [12]:
(1, 2) in H()

(1, 2)


False

In [13]:
isinstance('str', type)

False